In [1]:
import sys
print(sys.version)

3.10.10 (v3.10.10:aad5f6a891, Feb  7 2023, 08:47:40) [Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
pip install datasets kaggle PyMuPDF Pillow pytesseract opencv-python 


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Windows only - for loading the Kaggle.json file
# RUN THIS COMMAND ON TERMINAL --> TO CONNECT TO KAGGLE  OR RUN HERE WITH OS
import os

os.system(r'icacls "C:\Users\annap\.kaggle\kaggle.json" /inheritance:r /grant:r annap:R')


0

In [4]:
#Mac only - for loading the Kaggle.json file
# Assuming you have kaggle.json in Downloads
import os
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
os.system("cp ~/Downloads/kaggle.json ~/.kaggle/kaggle.json") #move from Downloads to .kaggle
os.system("chmod 600 ~/.kaggle/kaggle.json") #change permission to owner only


0

In [5]:
import kaggle
kaggle.api.authenticate()

from PIL import Image
import pytesseract
import pandas as pd
import numpy as np
import os
import cv2
import re

# from transformers import AutoProcessor, AutoModelForSequenceClassification, TrainingArguments, Trainer
from huggingface_hub import hf_hub_download
import torch
from transformers import LayoutLMv3ForSequenceClassification, LayoutLMv3Tokenizer, AutoProcessor
from datasets import Dataset  
import subprocess


/Users/makennawarner/StatsProject/document-classification/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/makennawarner/StatsProject/document-classification/venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/makennawarner/StatsProject/document-classification/venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2025-03-31 17:52:30.652853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is op

## From JPG


In [9]:
dataset_path = 'shaz13/real-world-documents-collections'
kaggle.api.dataset_download_files(dataset_path, path=dataset_path, unzip=True)
print(os.listdir(dataset_path))


Dataset URL: https://www.kaggle.com/datasets/shaz13/real-world-documents-collections
['docs-sm']


In [10]:
np.random.seed(500)
invoice_path = f'{dataset_path}/docs-sm/invoice'
print(os.listdir(invoice_path)) #lists the files in the invoice folder


['ti10161552.jpg', '2028697984.jpg', '2041158058.jpg', '0001219743.jpg', '2084024865.jpg', 'ti31689101.jpg', 'CTRCONTRACTS001598-1.jpg', '91661681.jpg', '93416592.jpg', '93216911.jpg', '80701185.jpg', '2030138144.jpg', '509592495_509592499.jpg', 'ti16352085.jpg', '80211559.jpg', '11238840.jpg', '2028717388.jpg', '2024525914.jpg', '03700650.jpg', 'tob04705.73_tob04705.76.jpg', '0060080760.jpg', '95824806.jpg', '2026004159.jpg', '92678319.jpg', '2041619314.jpg', '50120566-0566.jpg', '506271023.jpg', '2028716484.jpg', '2085669419.jpg', '2501335075.jpg', 'ti16310868.jpg', '2072957896.jpg', '85701956.jpg', '2048163101.jpg', '2026486780.jpg', '0000226471.jpg', '2072958016_8017.jpg', '0060031784.jpg', '01398469.jpg', '530003729+-3730.jpg', 'CTRCONTRACTS009895-9.jpg', '80233857.jpg', '2084061297.jpg', '2063610066.jpg', '2084061095.jpg', '2072484129.jpg', '2028712208.jpg', '2505407626.jpg', '83553602_3608.jpg', '0013122156.jpg', '95824354.jpg', '91507140.jpg', '89000889.jpg', 'ti10161636.jpg', 

# Utility Functions for LayoutMv3

In [11]:
#rescales bounding box coordinates to 1000x1000
def normalize_box(bbox, w, h):
    return [
        int(bbox[0]*(1000/w)),
        int(bbox[1]*(1000/h)),
        int(bbox[2]*(1000/w)),
        int(bbox[3]*(1000/h)),
    ]

#converts normalized bounding box coordinates to original image size
def unnormalize_box(bbox, w, h):
    return [
        w * (bbox[0] / 1000),
        hash * (bbox[1] / 1000),
        w * (bbox[2] / 1000),
        h * (bbox[3] / 1000),
    ]


#Compares bounding boxes - for debugging
def compare_boxes(b1, b2):
    b1 = np.array([c for c in b1])
    b2 = np.array([c for c in b2])
    equal = np.array_equal(b1, b2)
    return equal


#Checks if two consecutive words have the same label
def adjacent(w1, w2):
    if w1['label'] == w2['label'] and abs(w1['id'] - w2['id']) == 1:
      return True
    return False



# Preprocessing

In [12]:
'''tesseract runs OCR on images to recognize text'''
def run_tesseract_on_image(image_path, tsv_dir="tsv"):  # -> tsv output path
    # Convert the image_path to an absolute path relative to the current working directory
    full_image_path = os.path.join(os.getcwd(), image_path)
    full_image_path = os.path.normpath(full_image_path)
    print("Full image path:", full_image_path)
    
    # Extract the image name (without extension) for the output filename
    image_name = os.path.splitext(os.path.basename(full_image_path))[0]
    
    # Validate the image can be loaded
    img = cv2.imread(full_image_path)
    if img is None:
        raise ValueError(f"Unable to read image: {full_image_path}")
    else:
        print(f"Image '{full_image_path}' loaded successfully.")


    tsv_directory = os.path.join(os.getcwd(), "tsv")
    if not os.path.exists(tsv_directory):
        os.makedirs(tsv_directory)

    tsv_output_path = os.path.join(tsv_directory, f"{image_name}.tsv")
    tsv_output_path = os.path.normpath(tsv_output_path)

    print(f"Running Tesseract on image: {image_path}")
    print(f"Saving TSV to: {tsv_output_path}")

    # Tesseract needs a prefix (without .tsv extension) for output
    tsv_prefix = os.path.splitext(tsv_output_path)[0]
    
    # Run Tesseract (using the command available in PATH)
    result = subprocess.run(
        ["tesseract", full_image_path, tsv_prefix, "-l", "eng", "tsv"],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        print(f"Error running Tesseract: {result.stderr}")
        raise ValueError(f"Error running Tesseract on image '{full_image_path}'.")
    else:
        print("Tesseract completed successfully.")
        print("Tesseract output:", result.stdout)
        return tsv_output_path

In [13]:
'''reads the file generated above, removes empty/missing entries,
iterates through the cleaned dataset and stores words and bbox coordinates'''
def clean_tesseract_output(tsv_output_path):
    if not os.path.exists(tsv_output_path):
        raise FileNotFoundError(f"Error: The file {tsv_output_path} does not exist")
    
    print(f"Reading TSV file: {tsv_output_path}")
    try:
        # Use the Python engine and skip problematic lines
        ocr_df = pd.read_csv(tsv_output_path, sep='\t', engine='python', on_bad_lines='skip')
    except Exception as e:
        print("Error reading TSV file. File content:")
        with open(tsv_output_path, 'r', encoding='utf-8') as f:
            print(f.read())
        raise e

    ocr_df.dropna(inplace=True)
    if 'text' not in ocr_df.columns:
        raise ValueError("The TSV file does not contain a 'text' column. Check Tesseract output format.")
    
    # Convert 'text' column to string to avoid .str accessor errors
    ocr_df['text'] = ocr_df['text'].astype(str)
    ocr_df = ocr_df[ocr_df['text'].str.strip() != '']

    words = []
    for _, row in ocr_df.iterrows():
        origin_box = [
            row['left'],
            row['top'],
            row['left'] + row['width'],
            row['top'] + row['height']
        ]
        words.append({
            'word_text': row['text'],
            'word_box': origin_box
        })
    
    return words
   


In [14]:
def extract_text_bboxes(image_path, use_tesseract_tsv=True):
    if use_tesseract_tsv:                     #tesseract TSV
      tsv_path = run_tesseract_on_image(image_path)
      print("extract bboxes tsv_path: ", tsv_path)
      words_boxes = clean_tesseract_output(tsv_path)
      words = [item['word_text'] for item in words_boxes]
      bboxes = [item['word_box'] for item in words_boxes]

      print(f"Completed extract boxes function for: {image_path} ")

    return words, bboxes

In [15]:
#For mac- to ensure the script can find tesseract
os.environ["PATH"] += os.pathsep + "/usr/local/bin"


In [ ]:
def clean_text(df):
    """
    Explodes the 'tokens' column, removes empty tokens,
    and returns a list of dictionaries with word text and bounding boxes.
    """
    df = df.explode("tokens")
    df = df[df["tokens"].str.strip() != '']
    words = []
    for index, row in df.iterrows():
        word = {}
        origin_box = row['bboxes']
        word['word_text'] = row['tokens']
        word['word_box'] = origin_box
        words.append(word)
    return words

# --- Prepares for inference using relative paths ---
invoice_path = "shaz13/real-world-documents-collections/docs-sm/invoice"
invoice_data_list = []
print("Invoice path:", invoice_path)

for filename in os.listdir(invoice_path):
    if filename.lower().endswith(('.jpg', '.jpeg')):
        print(f"Valid image: {filename}")
        image_path = os.path.join(invoice_path, filename)
        print(f"Full image path before calling function: {image_path}")
        if os.path.exists(image_path):
            print(f"Image exists: {image_path}")
        else:
            print(f"Image does NOT exist: {image_path}")
        
        try:
            words, bboxes = extract_text_bboxes(image_path, use_tesseract_tsv=True)
            invoice_data_list.append({
                "file_name": filename,
                "tokens": words,
                "bboxes": bboxes,
            })
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    else:
        print(f"Skipping non-image file: {filename}")

df = pd.DataFrame(invoice_data_list)
df = clean_text(df)
print(df)


Invoice path: shaz13/real-world-documents-collections/docs-sm/invoice
Valid image: ti10161552.jpg
Full image path before calling function: shaz13/real-world-documents-collections/docs-sm/invoice/ti10161552.jpg
Image exists: shaz13/real-world-documents-collections/docs-sm/invoice/ti10161552.jpg
Full image path: /Users/makennawarner/StatsProject/document-classification/shaz13/real-world-documents-collections/docs-sm/invoice/ti10161552.jpg
Image '/Users/makennawarner/StatsProject/document-classification/shaz13/real-world-documents-collections/docs-sm/invoice/ti10161552.jpg' loaded successfully.
Running Tesseract on image: shaz13/real-world-documents-collections/docs-sm/invoice/ti10161552.jpg
Saving TSV to: /Users/makennawarner/StatsProject/document-classification/tsv/ti10161552.tsv
Tesseract completed successfully.
Tesseract output: 
extract bboxes tsv_path:  /Users/makennawarner/StatsProject/document-classification/tsv/ti10161552.tsv
Reading TSV file: /Users/makennawarner/StatsProject/do

LayoutMv3 needs textual and spatial information. So, we need the bounding boxes of where the text is positioned within the picture.

In [ ]:
from transformers import AutoProcessor

/Users/makennawarner/StatsProject/document-classification/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/makennawarner/StatsProject/document-classification/venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2025-03-31 14:39:03.831114: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

#convert into huggnig face dataset
hf_dataset = Dataset.from_pandas(df)
print(hf_dataset[0])   #check it worked


#passes words/bboxes into the layoutmv3 processer
def preprocess(example, img_path):
  img = Image.open(img_path).convert("RGB")
  encoding = processor(
      images=img,
      text=example["tokens"],
      boxes=example["bboxes"],
      truncation=True,
      padding="max_length"
  )
  return encoding


NameError: name 'df' is not defined

In [ ]:
# Function to add the image encoding to each example.
def add_image_encoding(example):
    # Construct the full path to the image using invoice_path and file_name
    file_path = os.path.join(invoice_path, example["file_name"])
    encoding = preprocess(example, file_path)
    # Update the example with the processor outputs
    example.update(encoding)
    return example

# Apply the preprocessing function over the entire dataset.
hf_dataset = hf_dataset.map(add_image_encoding)

In [ ]:
dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]
print("Example from train dataset:", train_dataset[0])


# Invoice Extraction

In [ ]:
# we will need to recreate this for all categories
def extract_invoice_data(text):
    data = {}

    data["Invoice Number"] = re.search(r'Invoice\s*No[:#]?\s*([\w-]+)', text, re.IGNORECASE)
    data["Invoice Date"] = re.search(r'Invoice\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Due Date"] = re.search(r'Due\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Issuer Name"] = re.search(r'From[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Recipient Name"] = re.search(r'To[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Total Amount"] = re.search(r'Total\s*Amount[:#]?\s*([\d,]+\.?\d{2})', text, re.IGNORECASE)

    for key, value in data.items():
        data[key] = value.group(1) if value else "Not Found"

    return data


# LayoutMv3

In [ ]:
#pretrained layoutmv3 model
model = AutoModelForSequenceClassification("microsoft/layoutlmv3-base", num_labels=4)

train_args = TrainingArguments(output_dir="./layoutlmv3_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs"
    )


trainer= Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
    )


trainer.train()



In [ ]:
#Example usage
# Load the fine-tuned model and processor
model = AutoModelForSequenceClassification.from_pretrained("./layoutlmv3_results")
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

# Define your inference function
def predict_invoice(image_path, tokens, bboxes):
    """
    Preprocesses the image along with tokens and bounding boxes,
    runs inference, and returns the predicted class.
    """
    img = Image.open(image_path).convert("RGB")
    encoding = processor(
        images=img,
        text=tokens,
        boxes=bboxes,
        return_tensors="pt",
        truncation=True,
        padding="max_length"
    )
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Use your OCR function to extract tokens and bounding boxes
# (Assuming extract_text_bboxes is already defined and working)
new_image_path = "shaz13/real-world-documents-collections/docs-sm/invoice/new_invoice.jpg"

# Extract tokens and bounding boxes from the image using Tesseract
tokens, bboxes = extract_text_bboxes(new_image_path, use_tesseract_tsv=True)

# Now pass these directly into your predict_invoice function
predicted = predict_invoice(new_image_path, tokens, bboxes)
print("Predicted class:", predicted)